### Install and Import Dependencies

In [46]:
import mediapipe as mp
import pickle
import numpy as np
import cv2
import csv
import pandas as pd
import os

The 33 keypoints are in order

0 - nose \
1 - left eye (inner) \
2 - left eye \
3 - left eye (outer) \
4 - right eye (inner) \
5 - right eye \
6 - right eye (outer) \
7 - left ear \
8 - right ear \
9 - mouth (left) \
10 - mouth (right) \
11 - left shoulder \
12 - right shoulder \
13 - left elbow \
14 - right elbow \
15 - left wrist \
16 - right wrist \
17 - left pinky \
18 - right pinky \
19 - left index \
20 - right index \
21 - left thumb \
22 - right thumb \
23 - left hip \
24 - right hip \
25 - left knee \
26 - right knee \
27 - left ankle \
28 - right ankle \
29 - left heel \
30 - right heel \
31 - left foot index \
32 - right foot index \

For squat detection, we don't need to the keypoints 1, 3, 4, and 5 (too much info about eyes), \
9 and 10 (don't need to know mouth position), and 17-22 (info about fingers too detailed, only \
need wrist keypoints)

### Capture Keypoints to CSV

In [67]:
def capture_pose(keypoints, filepath, classification):
    """
    Cleans keypoints data, writes file to csv.  

    :params:
        - keypoints: landmark obj of all 33 keypoints w/ x, y, z, and visibility
        - filepath: path of csv file to write to
        - classification: 1 if up position, 0 if down position
    """
    # clean up unwanted keypoints
    keypoints = keypoints[:1] + keypoints[2:3] + keypoints[6:9] + keypoints[11:17] + keypoints[23:33]
    # flatten to 1d vector
    cleaned_kp = np.array([[x.x, x.y, x.z] for x in keypoints], dtype="float32").flatten() 
    cleaned_kp = np.append(cleaned_kp, classification)
    cleaned_kp = cleaned_kp.reshape(1, len(cleaned_kp))
    df = pd.DataFrame(cleaned_kp)

    # write data to cvs
    file_exists = os.path.isfile(filepath)
    if not file_exists:
        df.to_csv(filepath, index=False)
    else:
        df.to_csv(filepath, mode='a', index=False, header=False)

### Capture Data to CSV

In [70]:
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# starts video capture on video
cap = cv2.VideoCapture('data\\test_videos\\Barbell Back Squat - Front View.mp4')
csv_filepath = "data\\squat_data.csv"

# runs mediapipe pose detection
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # changing display colors
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame.flags.writeable = False
        results = pose.process(frame)
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

        mp_drawing.draw_landmarks(
            frame,
            results.pose_landmarks, 
            mp_pose.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
            mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
            )

        cv2.imshow("Webcam Feed", frame)

        k = cv2.waitKey(1)
        if k == ord('u'):
            # captures keypoints for up position
            capture_pose(results.pose_landmarks.landmark, csv_filepath, 1)
        elif k == ord('d'):
            # captures keypoints for down position
            capture_pose(results.pose_landmarks.landmark, csv_filepath, 0)
        elif k == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()